In [133]:
import os 
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt

In [134]:
roundsPath = "data/matches/silver/rounds"
killsPath = "data/matches/silver/kills"
mapDataPath = "data/maps/map_data.csv"

map_name = "de_inferno"

In [135]:
map_data = pd.read_csv(mapDataPath)
map_data.rename(columns={'Unnamed: 0' : 'map'},inplace=True)

idx_ref = map_data.columns
cl_ref = map_data['map'].values

map_data.drop(columns='map', inplace=True)

map_data = map_data.T
map_data.columns = cl_ref

In [136]:
startX = map_data[map_name][-2]
endX = map_data[map_name][0]
resX = map_data[map_name][2]

startY = map_data[map_name][-1]
endY = map_data[map_name][1]
resY = map_data[map_name][3]

def pointx_to_resolutionx(xinput,startX=startX,endX=endX,resX=resX):
    sizeX=endX-startX
    if startX < 0:
        xinput += startX *(-1.0)
    else:
        xinput += startX
    xoutput = float((xinput / abs(sizeX)) * resX);
    return xoutput

def pointy_to_resolutiony(yinput,startY=startY,endY=endY,resY=resY):
    sizeY=endY-startY
    if startY < 0:
        yinput += startY *(-1.0)
    else:
        yinput += startY
    youtput = ((yinput / abs(sizeY)) * resY);
    return resY-youtput

In [137]:
rounds = pq.ParquetDataset(roundsPath).read().to_pandas()

In [138]:
kills = pq.ParquetDataset(killsPath).read().to_pandas()

In [139]:
df = kills.merge(rounds, right_on=['id','round_number'],
           left_on=['id','round_number'])

In [140]:
df = df.copy()[['id', 'killer_side', 'killer_team', 'killed_side', 'killed_team',
       'killer_name', 'killed_name', 'assister_name', 'killer_x', 'killer_y',
       'round_number', 'victim_x', 'victim_y', 'seconds', 'element', 'type',
       'weapon_name', 'time_death_seconds', 'map_name',
       'end_reason', 'winner_side', 'winner_name',
       'team_t_name', 'team_ct_name', 'equipement_value_team_t',
       'equipement_value_team_ct', 'start_money_team_t', 'start_money_team_ct',
       'duration']]

## Keys dropped:
# non-valuable features: all counts, tick
# data leakage: kills_count

In [141]:
df.columns

Index(['id', 'killer_side', 'killer_team', 'killed_side', 'killed_team',
       'killer_name', 'killed_name', 'assister_name', 'killer_x', 'killer_y',
       'round_number', 'victim_x', 'victim_y', 'seconds', 'element', 'type',
       'weapon_name', 'time_death_seconds', 'map_name', 'end_reason',
       'winner_side', 'winner_name', 'team_t_name', 'team_ct_name',
       'equipement_value_team_t', 'equipement_value_team_ct',
       'start_money_team_t', 'start_money_team_ct', 'duration'],
      dtype='object')

In [142]:
## Fixing column names for FURIA case 
# Consider this step to be done for the others teams as well (currently=MIBR and EG)

df['team_ct_name'] = df['team_ct_name'].apply(lambda x: x if "FURIA" not in x else "FURIA") \
    .apply(lambda y: y if "Evil" not in y else "EG") \
    .apply(lambda z: z if z not in ["mibr-","made in brazil"] else "MIBR").str.strip()

df['team_t_name'] = df['team_t_name'].apply(lambda x: x if "FURIA" not in x else "FURIA") \
    .apply(lambda y: y if "Evil" not in y else "EG") \
    .apply(lambda z: z if z not in ["mibr-","made in brazil"] else "MIBR").str.strip()

df['killer_team'] = df['team_ct_name'].apply(lambda x: x if "FURIA" not in x else "FURIA") \
    .apply(lambda y: y if "Evil" not in y else "EG") \
    .apply(lambda z: z if z not in ["mibr-","made in brazil"] else "MIBR").str.strip()

df['killed_team'] = df['team_t_name'].apply(lambda x: x if "FURIA" not in x else "FURIA") \
    .apply(lambda y: y if "Evil" not in y else "EG") \
    .apply(lambda z: z if z not in ["mibr-","made in brazil"] else "MIBR").str.strip()

df['winner_name'] = df['team_t_name'].apply(lambda x: x if "FURIA" not in x else "FURIA") \
    .apply(lambda y: y if "Evil" not in y else "EG") \
    .apply(lambda z: z if z not in ["mibr-","made in brazil"] else "MIBR").str.strip()

# Creating binary target for rounds that FURIA has won

df['winner'] = df['winner_name'].apply(lambda x:
                                           1 if x=="FURIA" else 0)
# Defining binary target for sides

df['killer_side'] = df['killer_side'].apply(lambda x: 1 if x=="CT" else 0)

df['killed_side'] = df['killed_side'].apply(lambda x: 1 if x=="CT" else 0)

df['winner_side'] = df['killed_side'].apply(lambda x: 1 if x=="CT" else 0)

# Creating binary target for kills that has assists

df['has_assist'] = df['assister_name'].apply(lambda x: 0 if x == None else 1)

# Scalling coordinates 

coordinatedColumns =  ['killer_x', 'killer_y','victim_x','victim_y']
for cl in coordinatedColumns:
    if 'x' in cl:
        df[cl] = df[cl].apply(lambda x: pointx_to_resolutionx(x))
    else:
        df[cl] = df[cl].apply(lambda y: pointy_to_resolutiony(y))

In [143]:
for column in df.columns:
    if(df[column].dtype == 'O'):
        df[column] = df[column].apply(lambda y: y.strip() if type(y) == str else y) \
        .apply(lambda x: None if x == "" else x)

In [144]:
# Dropping columns that won't scale for the modelling

df.drop(columns=['id',"assister_name"], inplace=True)
df.dropna(inplace=True)

# Creating a new slice by selecting a single map

df = df.copy().loc[df["map_name"] == map_name]
df.reset_index(drop=True, inplace=True)

In [189]:
# Splitting features by X, y

X = df.copy().drop(columns=['winner','map_name','winner_name','winner_side','end_reason'])
y = df.copy()['winner']

### Feature Treatment

In [176]:
to_transform = []

for column in X.columns:
    if(X[column].dtype == 'O'):
        to_transform.append(column)

In [177]:
inverse_transform = {}

In [178]:
for column in to_transform:
    labelEncoder = pd.Series(X[column].unique()).reset_index().set_index(0).to_dict()['index']
    
    ## Creating and storing inverseEncode
    inverseEncoder = pd.Series(df[column].unique()).reset_index() \
    .rename(columns={0:column}).set_index('index').to_dict()[column]
    
    inverse_transform[column] = inverseEncoder
    
    # Mapping label encoding
    X[column] = X[column].map(labelEncoder).astype(int)

### Modelling

In [179]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_tree

In [180]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=0)

In [181]:
xgb = XGBClassifier(use_label_encoder=False)
gbc = GradientBoostingClassifier()

In [182]:
xgb.fit(X_train, y_train)

[19:16:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=6, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [183]:
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

In [184]:
feature_importance = pd.DataFrame(
    gbc.feature_importances_,index=X.columns,columns=['importance']) \
    .sort_values(by='importance', ascending=False)

### Evaluating

In [200]:
xgb_train_score, xgb_test_score = xgb.score(X_train, y_train), xgb.score(X_test, y_test)

In [201]:
gbc_train_score, gbc_test_score = gbc.score(X_train, y_train), gbc.score(X_test, y_test)

In [208]:
print("*XGBoost*\n-> train:{:.2f}\n-> test:{:.2f}\n\n*GBC\n-> train:{:.2f}\n-> test:{:.2f}".
      format(xgb_train_score, xgb_test_score,gbc_train_score, gbc_test_score ))

*XGBoost*
-> train:1.00
-> test:0.99

*GBC
-> train:0.93
-> test:0.89


In [158]:
evaluate = X_test.copy()
evaluate['pred'] = y_test.copy()
evaluate['win_chance'] = [x[1] for x in xgb.predict_proba(X_test)]

In [159]:
for col in to_transform:
    evaluate[col] = evaluate[col].map(inverse_transform[col])

In [160]:
evaluate.loc[(evaluate['pred'] == 1)]

,killer_side,killer_team,killed_side,killed_team,killer_name,killed_name,killer_x,killer_y,round_number,victim_x,...,team_t_name,team_ct_name,equipement_value_team_t,equipement_value_team_ct,start_money_team_t,start_money_team_ct,duration,has_assist,pred,win_chance
940,0,EG,1,FURIA,KSCERATO,Ethan,578.035170,250.587002,29,704.290468,...,FURIA,EG,19700,30600,19550,22800,183.27,1,1,0.998817
565,1,Isurus,0,FURIA,JonY BoY,VINI,526.707655,72.380657,27,519.301621,...,FURIA,Isurus,25150,31950,24650,28550,183.95,1,1,0.998817
1412,1,OGEsports,0,FURIA,arT,Aleksib,626.300098,214.793244,21,559.606711,...,FURIA,OGEsports,23450,14500,22050,9750,113.96,0,1,0.998817
37,0,GenG-,1,FURIA,VINI,koosta,815.864384,739.183271,6,848.863097,...,FURIA,GenG-,25900,24900,38650,15200,137.50,1,1,0.998817
878,0,EG,1,FURIA,HEN1,tarik,430.323632,664.018360,18,509.335727,...,FURIA,EG,22150,1200,20500,12750,67.26,0,1,0.998817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,0,Team Liquid,1,FURIA,yuurih,NAF,589.568018,356.853259,14,532.522375,...,FURIA,Team Liquid,26100,8300,57550,18700,75.07,0,1,0.998817
1816,0,Team Liquid,1,FURIA,arT,Twistzz,822.849248,180.427031,9,932.828884,...,FURIA,Team Liquid,25200,29300,41250,22550,105.09,1,1,0.998817
916,0,EG,1,FURIA,Yuurih,CeRq,867.520076,565.163719,25,979.654036,...,FURIA,EG,13600,30500,11500,19000,166.70,0,1,0.998817
1433,0,OGEsports,1,FURIA,valde,arT,835.195883,738.840220,24,947.418275,...,FURIA,OGEsports,23800,31650,22900,23800,128.03,0,1,0.998817
